# TP 1: LDA/QDA y optimización matemática de modelos

# Intro teórica

## Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

## Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

## LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

## Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

## Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

In [1]:
!pip install -q numpy pandas matplotlib scipy scikit-learn==1.6.1


In [4]:
!touch amia_tp1/base/__init__.py
!touch amia_tp1/utils/__init__.py


In [6]:
import sys
sys.path.append('/content/amia_tp1')


# Código provisto

Con el fin de no retrasar al alumno con cuestiones estructurales y/o secundarias al tema que se pretende tratar, se provee una base de código que **no es obligatoria de usar** pero se asume que resulta resulta beneficiosa.

In [7]:
# imports
import numpy as np
import numpy.linalg as LA

from base.qda import QDA, TensorizedQDA
from base.cholesky import QDA_Chol1, QDA_Chol2, QDA_Chol3
from utils.bench import Benchmark
from utils.datasets import (get_iris_dataset, get_letters_dataset,
                            get_penguins_dataset, get_wine_dataset,
                            label_encode)

## Base code

## Datasets

Observar que se proveen **4 datasets diferentes**, el código de ejemplo usa uno solo pero eso no significa que ustedes se limiten al mismo. También pueden usar otros datasets de su elección.

## Benchmarking

Nota: esta clase fue creada bastante rápido y no pretende ser una plataforma súper confiable sobre la que basarse, sino más bien una herramienta simple con la que poder medir varios runs y agregar la información.

En forma rápida, `warmup` es la cantidad de runs para warmup, `mem_runs` es la cantidad de runs en las que se mide el pico de uso de RAM y `n_runs` es la cantidad de runs en las que se miden tiempos.

La razón por la que se separan es que medir memoria hace ~2.5x más lento cada run, pero al mismo tiempo se estabiliza mucho más rápido.

**Importante:** tener en cuenta que los modelos que predicen en batch (usan `predict` directamente) deberían consumir, como mínimo, $n$ veces la memoria de los que predicen por observación.

## Ejemplo

In [8]:
# levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total
X_full, y_full = get_wine_dataset()

X_full.shape, y_full.shape

((178, 13), (178, 1))

In [9]:
# encodeamos a número las clases
y_full_encoded = label_encode(y_full)

y_full[:5], y_full_encoded[:5]

(array([['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0']], dtype='<U7'),
 array([[0],
        [0],
        [0],
        [0],
        [0]]))

In [10]:
# generamos el benchmark
# observar que son valores muy bajos de runs para que corra rápido ahora
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 100,
    warmup = 20,
    mem_runs = 20,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 140
Warmup runs: 20
Peak Memory usage runs: 20
Running time runs: 100
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [89]:
# bencheamos un par
to_bench = [QDA]

for model in to_bench:
    b.bench(model)

QDA (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

QDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [90]:
# como es una clase, podemos seguir bencheando más después
b.bench(TensorizedQDA)

TensorizedQDA (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

TensorizedQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [96]:
class FasterQDA(TensorizedQDA):
    def fit(self, X, y):
        # y como vector plano
        y = y.flatten() if y.ndim > 1 else y

        # priors en log
        counts = np.bincount(y)
        self.log_a_priori = np.log(counts / counts.sum())

        # parámetros del modelo (llena inv_covs/means y sus stacks tensoriales)
        self._fit_params(X, y)

    def predict(self, X):
        # Convención del TP: X tiene forma (p, n)
        p, n = X.shape

        # 1) Restamos medias por clase a TODAS las observaciones a la vez
        #    self.tensor_means: (K, p, 1)  -> broadcast con X[None, :, :] (1, p, n)
        diffs = X[None, :, :] - self.tensor_means   # -> (K, p, n)

        # 2) Forma cuadrática: (x-μ)^T Σ^{-1} (x-μ) en bloque
        #    inv: (K, p, p)  y diffs: (K, p, n)
        #    resultado intermedio: (K, p, n)
        left = np.einsum('kpq,kqn->kpn', self.tensor_inv_cov, diffs)

        #    producto elemento a elemento y suma en "p" -> (K, n)
        quad = np.einsum('kpn,kpn->kn', diffs, left)

        # 3) Términos extra por clase
        log_dets = np.log(np.linalg.det(self.tensor_inv_cov))      # (K,)
        scores = (-0.5 * quad - 0.5 * log_dets[:, None] + self.log_a_priori[:, None]).T  # (n, K)

        # 4) Elegimos la clase ganadora por observación
        return np.argmax(scores, axis=1).reshape(-1, 1)


In [97]:
b.bench(FasterQDA)


FasterQDA (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

FasterQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [98]:
class EfficientQDA(TensorizedQDA):
    def fit(self, X, y):
        # y vector plano
        y = y.flatten() if y.ndim > 1 else y
        # log-priors por clase
        counts = np.bincount(y)
        self.log_a_priori = np.log(counts / counts.sum())
        # parámetros (inv covs y medias apiladas)
        self._fit_params(X, y)
        # opcional
        self.n_classes_ = len(self.log_a_priori)
        return self

    def predict(self, X):
        # *** Convención del TP: X debe ser (p, n) ***
        # Si viniera como (n, p), lo giro para que p coincida con self.tensor_means
        p_expected = self.tensor_means.shape[1]  # (K, p, 1)
        if X.shape[0] != p_expected:
            X = X.T  # ahora X es (p, n)

        # diffs: (K, p, n)  ← restamos medias por clase a TODAS las observaciones
        diffs = X[None, :, :] - self.tensor_means   # (K, p, n)

        # Aplicamos Σ^{-1} sin formar n×n:
        # left = Σ^{-1} * diffs  → (K, p, n)
        left = np.einsum('kpq,kqn->kpn', self.tensor_inv_cov, diffs)

        # Forma cuadrática por clase y observación: sumo en la dimensión p
        # quad[k, i] = sum_p diffs[k,p,i] * left[k,p,i]
        quad = np.einsum('kpn,kpn->kn', diffs, left)  # (K, n)

        # Términos por clase
        log_dets = np.log(np.linalg.det(self.tensor_inv_cov))      # (K,)
        scores = (-0.5 * quad
                  - 0.5 * log_dets[:, None]
                  + self.log_a_priori[:, None])                    # (K, n)

        # Ganadora por observación
        return np.argmax(scores, axis=0).reshape(-1, 1)


In [99]:
b.bench(EfficientQDA)


EfficientQDA (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

EfficientQDA (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [106]:
from base.cholesky import QDA_Chol1, QDA_Chol2, QDA_Chol3

b.bench(QDA_Chol1)
b.bench(QDA_Chol2)
b.bench(QDA_Chol3)



QDA_Chol1 (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

QDA_Chol1 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

QDA_Chol2 (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

QDA_Chol2 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

QDA_Chol3 (MEM):   0%|          | 0/20 [00:00<?, ?it/s]

QDA_Chol3 (TIME):   0%|          | 0/100 [00:00<?, ?it/s]

In [107]:
# hacemos un summary
b.summary()

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb
model,,,,,,,,,
FasterQDA,0.455744,0.167294,0.128970,0.128546,0.941111,0.019135,0.000880,0.230565,0.000000
QDA,0.506702,0.461892,3.159476,1.198602,0.982593,0.018429,0.000659,0.007899,0.000161
TensorizedQDA,0.486965,0.208660,1.395824,0.507763,0.985556,0.017982,0.000904,0.011993,0.000191
EfficientQDA,0.474626,0.203319,0.138417,0.135164,0.934444,0.019135,0.000648,0.230565,0.000000
QDA_Chol1,0.587004,0.301749,1.894843,0.581547,0.980926,0.018448,0.000892,0.008171,0.000074
QDA_Chol2,0.646373,0.487691,7.237625,1.564957,0.980926,0.018692,0.000892,0.008610,0.000292
QDA_Chol3,0.524516,0.373794,1.930844,0.707615,0.982222,0.017853,0.000751,0.008171,0.000025


In [108]:
# son muchos datos! nos quedamos con un par nomás
summ = b.summary()

# como es un pandas DataFrame, subseteamos columnas fácil
summ[['train_median_ms', 'test_median_ms','mean_accuracy']]

,train_median_ms,test_median_ms,mean_accuracy
model,,,
FasterQDA,0.455744,0.128970,0.941111
QDA,0.506702,3.159476,0.982593
TensorizedQDA,0.486965,1.395824,0.985556
EfficientQDA,0.474626,0.138417,0.934444
QDA_Chol1,0.587004,1.894843,0.980926
QDA_Chol2,0.646373,7.237625,0.980926
QDA_Chol3,0.524516,1.930844,0.982222


In [111]:
# podemos setear un baseline para que fabrique columnas de comparación
summ = b.summary(baseline='QDA')

summ

,train_median_ms,train_std_ms,test_median_ms,test_std_ms,mean_accuracy,train_mem_median_mb,train_mem_std_mb,test_mem_median_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
FasterQDA,0.455744,0.167294,0.128970,0.128546,0.941111,0.019135,0.000880,0.230565,0.000000,1.111813,24.497755,0.963118,0.034258
QDA,0.506702,0.461892,3.159476,1.198602,0.982593,0.018429,0.000659,0.007899,0.000161,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,0.486965,0.208660,1.395824,0.507763,0.985556,0.017982,0.000904,0.011993,0.000191,1.040530,2.263520,1.024820,0.658596
EfficientQDA,0.474626,0.203319,0.138417,0.135164,0.934444,0.019135,0.000648,0.230565,0.000000,1.067582,22.825776,0.963118,0.034258
QDA_Chol1,0.587004,0.301749,1.894843,0.581547,0.980926,0.018448,0.000892,0.008171,0.000074,0.863200,1.667408,0.998966,0.966678
QDA_Chol2,0.646373,0.487691,7.237625,1.564957,0.980926,0.018692,0.000892,0.008610,0.000292,0.783916,0.436535,0.985893,0.917424
QDA_Chol3,0.524516,0.373794,1.930844,0.707615,0.982222,0.017853,0.000751,0.008171,0.000025,0.966038,1.636318,1.032265,0.966678


In [112]:
summ[[
    'train_median_ms', 'test_median_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

,train_median_ms,test_median_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
FasterQDA,0.455744,0.128970,0.941111,1.111813,24.497755,0.963118,0.034258
QDA,0.506702,3.159476,0.982593,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,0.486965,1.395824,0.985556,1.040530,2.263520,1.024820,0.658596
EfficientQDA,0.474626,0.138417,0.934444,1.067582,22.825776,0.963118,0.034258
QDA_Chol1,0.587004,1.894843,0.980926,0.863200,1.667408,0.998966,0.966678
QDA_Chol2,0.646373,7.237625,0.980926,0.783916,0.436535,0.985893,0.917424
QDA_Chol3,0.524516,1.930844,0.982222,0.966038,1.636318,1.032265,0.966678


In [114]:
#ordenado por tiempo
summ.sort_values('test_median_ms')[['train_median_ms','test_median_ms','mean_accuracy']]


,train_median_ms,test_median_ms,mean_accuracy
model,,,
FasterQDA,0.455744,0.128970,0.941111
EfficientQDA,0.474626,0.138417,0.934444
TensorizedQDA,0.486965,1.395824,0.985556
QDA_Chol1,0.587004,1.894843,0.980926
QDA_Chol3,0.524516,1.930844,0.982222
QDA,0.506702,3.159476,0.982593
QDA_Chol2,0.646373,7.237625,0.980926


# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda *fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?
2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.
4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.
5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
$$
np.sum(A^T \odot B, axis=0).T
$$
queda a preferencia del alumno cuál usar.
6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.
7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?
7. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.
8. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?
9. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.
13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.
13. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Importante:

Las métricas que se observan al realizar benchmarking son muy dependientes del código que se ejecuta, y por tanto de las versiones de las librerías utilizadas. Una forma de unificar esto es utilizando un gestor de versiones y paquetes como _uv_ o _Poetry_, otra es simplemente usando una misma VM como la que provee Colab.

**Cada equipo debe informar las versiones de Python, NumPy y SciPy con que fueron obtenidos los resultados. En caso de que sean múltiples, agregar todos los casos**. La siguiente celda provee una ayuda para hacerlo desde un notebook, aunque como es una secuencia de comandos también sirve para consola.

In [17]:
# Version que deberia funcionar en todos los entornos

import sys
print("Python:", sys.version)

# importlib.metadata es estándar desde Python 3.8; en Colab también está.
try:
    from importlib.metadata import version, PackageNotFoundError
except Exception:
    from importlib_metadata import version, PackageNotFoundError  # backport, por si acaso

for pkg in ["numpy", "scipy"]:
    try:
        print(f"{pkg}:", version(pkg))
    except PackageNotFoundError:
        print(f"{pkg}: NO INSTALADO")

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
numpy: 2.0.2
scipy: 1.16.1
